<a href="https://colab.research.google.com/github/CSID-DGU/2024-2-DSCD-Fiveguys-6/blob/main/%EC%9C%A0%EC%A0%84%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98_%EC%B6%9C%EA%B7%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install osmnx

In [ ]:
!pip install deap

In [ ]:
import osmnx as ox
import networkx as nx
import random
from deap import base, creator, tools, algorithms
import math

In [ ]:
# 후보 정류장 좌표 (예: {정류장 번호: (위도, 경도)}) 출근
stop_coordinates = {
    1: (126.913730, 37.643407),
    2: (126.919017, 37.631088),
    3: (126.918989,	37.636768),
    4: (126.918577,	37.637248),
    5: (126.931529,	37.633348),
    6: (126.931007,	37.631383),
    7: (126.934438,	37.636736),
    8: (126.923090,	37.643408),
    9: (126.930479,	37.648435),
    10:(126.928970,	37.644075),
    11:(126.921548,	37.641157),
    12:(126.927130,	37.645633),
    13:(126.929249,	37.634314),
    14:(126.926262, 37.634267),
    15:(126.921336,	37.644080),

    16: (126.972855, 37.555479),
    17: (126.967980, 37.566885),
    18: (126.919996, 37.625283),
    19: (126.920934, 37.619323),
    20: (126.925090, 37.615607),
    21: (126.918989, 37.636768),
    22: (126.918577, 37.637248),
    23: (126.919996, 37.617821),
    24: (126.924380, 37.620520),
    25: (126.922534, 37.619597),
    26: (126.934226, 37.600872),
    27: (126.962533, 37.569939),
    28: (126.965830, 37.566475),
    29: (126.969224, 37.562476),
    30: (126.924316, 37.557326)
    # 필요에 따라 추가 가능
}

In [ ]:
# 각 정류장별 대중교통 수요 (예: {정류장 번호: 수요 값}) 출근
stop_demand ={
    1: 4351,
    2: 6777,
    3: 7726,
    4: 7100,
    5: 2904,
    6: 2767,
    7: 5883,
    8: 3513,
    9: 6158,
    10:3247,
    11:7117,
    12:7041,
    13:3290,
    14:3610,
    15:3662,

    16: 20449,
    17: 15811,
    18: 5066,
    19: 12609,
    20: 5466,
    21: 8772,
    22: 18999,
    23: 20715,
    24: 6312,
    25: 14539,
    26: 36335,
    27: 19854,
    28: 21028,
    29: 18610,
    30: 26520
    # 필요에 따라 추가 가능
}

In [ ]:
# 도로 네트워크 불러오기 (서울시 예제)
G = ox.graph_from_place('Seoul, South Korea', network_type='drive')

In [ ]:
import matplotlib.pyplot as plt

# 도로 네트워크 시각화
fig, ax = ox.plot_graph(G, node_size=5, edge_color='gray', edge_linewidth=0.5, show=True, close=False)
plt.show()

In [ ]:
# OSMnx 네트워크에서 가장 가까운 노드 찾기
def get_nearest_node(coord):
    return ox.distance.nearest_nodes(G, coord[1], coord[0])

# 정류장 간 최단 경로 거리 계산 함수
def road_distance(coord1, coord2):
    node1 = get_nearest_node(coord1)
    node2 = get_nearest_node(coord2)
    try:
        return nx.shortest_path_length(G, node1, node2, weight='length') / 1000  # 단위: km
    except nx.NetworkXNoPath:
        return float('inf')  # 경로가 없는 경우 매우 큰 값 반환

# 수요 재분배 함수
def redistribute_demand(removed_stop, current_route):
    removed_coord = stop_coordinates[removed_stop]
    total_demand = stop_demand[removed_stop]
    demand_distribution = {}

    for stop in current_route:
        stop_coord = stop_coordinates[stop]
        distance = road_distance(removed_coord, stop_coord)
        if distance < float('inf'):  # 경로가 존재할 때만
            weight = max(0, 1 - (distance / 2))  # 2km 내에서 가중치 설정
            demand_distribution[stop] = total_demand * weight

    return {stop: demand for stop, demand in demand_distribution.items() if demand > 0}

def evaluate_route_with_time_and_demand(individual):
    total_distance = 0
    total_demand = 0
    total_stop_time = 0

    # 기본 정차 시간 및 승하차 시간 설정
    stop_time_per_stop = 1  # 정차 시간 (분)
    load_unload_time_per_passenger = 0.05  # 승하차 시간 (1명당 0.05분)

    for i in range(len(individual) - 1):
        stop1 = stop_coordinates[individual[i]]
        stop2 = stop_coordinates[individual[i + 1]]
        distance = road_distance(stop1, stop2)

        if distance == float('inf'):
            return float('inf'),  # 경로가 없을 경우 매우 낮은 적합도 반환

        total_distance += distance
        stop_demand_value = stop_demand.get(individual[i], 0)
        total_demand += stop_demand_value

        # 정차 시간 및 승하차 시간 계산
        total_stop_time += stop_time_per_stop
        total_stop_time += stop_demand_value * load_unload_time_per_passenger

    # 마지막 정류장 처리
    last_stop_demand = stop_demand.get(individual[-1], 0)
    total_stop_time += stop_time_per_stop
    total_stop_time += last_stop_demand * load_unload_time_per_passenger
    total_demand += last_stop_demand

    # 정류장 제거 및 수요 재분배
    if random.random() < 0.1:  # 정류장 제거 확률
        removed_stop = random.choice(individual)
        new_route = individual[:]
        new_route.remove(removed_stop)
        redistributed_demand = redistribute_demand(removed_stop, new_route)

        # 재분배된 수요 반영
        for stop, demand in redistributed_demand.items():
            total_demand += demand

    # 스케일링된 적합도 계산
    # 각 요소의 가중치를 설정하여 조정 (예제: 수요에 더 큰 가중치 부여)
    demand_weight = 1.0
    stop_time_weight = 0.5
    distance_weight = 0.3

    fitness = (demand_weight * total_demand) - (stop_time_weight * total_stop_time) - (distance_weight * total_distance)
    return fitness,  # 튜플로 반환

In [ ]:
# DEAP 초기화
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # 거리 최소화 및 수요 반영
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_stop", lambda: random.choice(list(stop_coordinates.keys())))
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_stop, n=30)  # 정류장 수 설정
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate_route_with_time_and_demand)

In [ ]:
# 유전 알고리즘 실행
population = toolbox.population(n=100)
NGEN = 50  # 세대 수
CXPB, MUTPB = 0.7, 0.2  # 교차와 변이 확률

for gen in range(NGEN):
    print(f"Generation {gen + 1}")  # 현재 세대 출력

    # 선택 및 복제
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    # 교차 연산
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            if len(child1) == len(child2):  # 개체 길이 확인
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
            else:
                print("Skipping crossover due to mismatched lengths")

    # 변이 연산
    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
        del mutant.fitness.values

    # 적합도 재평가
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # 세대 교체
    population[:] = offspring

    # 현재 세대의 최적 결과 출력
    best_individual = tools.selBest(population, 1)[0]
    print(f"  Best Individual: {best_individual}")
    print(f"  Fitness: {best_individual.fitness.values[0]}")

# 최적 노선 출력
best_individual = tools.selBest(population, 1)[0]
print("최적 노선:", best_individual)
print("적합도(총 거리 - 수요):", best_individual.fitness.values[0])